<a href="https://colab.research.google.com/github/PratiteeMalakar/hello-world/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 4.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

# Generate sample data for demand forecasting
dates = pd.date_range(start='2023-01-01', periods=365, freq='D')
products = ['Product_A', 'Product_B', 'Product_C']

data = []
for product in products:
    sales = np.random.poisson(lam=20, size=len(dates))
    data.extend(zip([product]*len(dates), dates, sales))

df_sales = pd.DataFrame(data, columns=['Product', 'Date', 'Sales'])
df_sales.to_csv('sales_data.csv', index=False)

# Generate sample data for inventory levels
inventory_data = {'Product': products, 'Inventory_Level': [100, 150, 200]}
df_inventory = pd.DataFrame(inventory_data)
df_inventory.to_csv('inventory_data.csv', index=False)

# Generate sample data for logistics
logistics_data = {
    'Product': products,
    'Warehouse': ['W1', 'W2', 'W3'],
    'Transportation_Cost': [10, 15, 20]
}
df_logistics = pd.DataFrame(logistics_data)
df_logistics.to_csv('logistics_data.csv', index=False)


In [3]:
from pinecone import Pinecone
import os
from google.colab import userdata

os.environ["PINECONE_API_KEY"]=userdata.get("PINECONE")

api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key = api_key)

In [4]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud , region=region)

In [13]:
import time
index_name = "index4"

pc.create_index(index_name , dimension=384, metric='cosine', spec=spec)



while not pc.describe_index(index_name).status['ready']:
  time.sleep(1)

In [14]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Load and encode knowledge base
model = SentenceTransformer('all-MiniLM-L6-v2')
df_sales = pd.read_csv('sales_data.csv')
df_inventory = pd.read_csv('inventory_data.csv')
df_logistics = pd.read_csv('logistics_data.csv')

knowledge_base = pd.concat([df_sales, df_inventory, df_logistics]).reset_index(drop=True)
sentences = knowledge_base.apply(lambda row: ' '.join(row.values.astype(str)), axis=1).tolist()
embeddings = model.encode(sentences)

# Connect to the index
index = pc.Index(index_name)

# Add items to the index
for i, embedding in enumerate(embeddings):
    index.upsert([(str(i), embedding)])

print("Indexing completed.")


Indexing completed.


In [8]:
pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [15]:
from transformers import pipeline

# Initialize the generative model
generator = pipeline('text-generation', model='gpt-3.5-turbo')

def retrieve_and_generate(query):
    # Encode the query
    query_embedding = model.encode([query])[0]

    # Retrieve top-5 similar documents
    results = index.query(queries=[query_embedding], top_k=5)
    retrieved_docs = [sentences[int(match['id'])] for match in results['matches']]

    # Generate a response using the retrieved documents as context
    context = ' '.join(retrieved_docs)
    response = generator(f'Context: {context} Question: {query}', max_length=100)

    return response[0]['generated_text']

# Example usage
query = "What is the optimal inventory level for Product_A?"
response = retrieve_and_generate(query)
print(response)


OSError: gpt-3.5-turbo is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`